In [1]:
import sys
sys.path.append("/home/brunoperdigao/repos/Plan-Classification-Topologic-DGL")

In [2]:
# Import Libraries
from topologicpy.DGL import DGL
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
import dgl
import torch
import random
from dgl import save_graphs, load_graphs

/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL/.venv/lib/python3.12/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [3]:
import dgl

In [6]:
pwd

'/home/bruno_perdigao/repos/Plan-Classification-Topologic-DGL'

In [9]:
path = "./data"
dataset = DGL.DatasetByCSVPath(path, numberOfGraphClasses=0,
                           nodeATTRKey='feat', edgeATTRKey='feat',
                           nodeOneHotEncode=False, nodeFeaturesCategories=[],
                           edgeOneHotEncode=False, edgeFeaturesCategories=[], addSelfLoop=False)
print(dataset)

Done saving data into cached files.
Dataset("GraphDGL", num_graphs=5, save_path=/home/bruno_perdigao/.dgl/GraphDGL)


In [10]:
train_data, val_data, test_data = dgl.data.utils.split_dataset(dataset, [0.7, 0.1, 0.2])

In [11]:
train_data.dataset

Dataset("GraphDGL", num_graphs=5, save_path=/home/bruno_perdigao/.dgl/GraphDGL)

---

In [12]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [13]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [14]:
# # print(len(dataset.graphs))
# for i, graph in enumerate(dataset.graphs):
#     # print(i)
#     node_features = graph.ndata['feat']
#     # print(node_features)
#     node_labels = graph.ndata['label']
#     # print(node_labels)
#     train_mask = graph.ndata['train_mask']
#     valid_mask = graph.ndata['val_mask']
#     test_mask = graph.ndata['test_mask']
#     n_features = node_features.shape[1]
#     # print(n_features)
#     n_labels = int(node_labels.max().item() + 1)
#     # print(n_features)

In [15]:
n_features = 2
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=11)
opt = torch.optim.Adam(model.parameters())

for epoch in range(50):
    for i, graph in enumerate(dataset.graphs):
        node_features = graph.ndata['feat']
        node_labels = graph.ndata['label']
        train_mask = graph.ndata['train_mask']
        valid_mask = graph.ndata['val_mask']
        test_mask = graph.ndata['test_mask']
        n_features = node_features.shape[1]
        n_labels = int(node_labels.max().item() + 1)
        
        model.train()
        # forward propagation by using all nodes
        logits = model(graph, node_features)
        # compute loss
        loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
        # compute validation accuracy
        acc = evaluate(model, graph, node_features, node_labels, valid_mask)
        # backward propagation
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
    
        # Save model if necessary.  Omitted in this example.

44.228824615478516
50.97963333129883
49.24543380737305
39.38355255126953
49.81715774536133
32.593204498291016
39.81265640258789
39.666358947753906
30.78075408935547
41.43587112426758
25.37822914123535
32.75198745727539
33.35314178466797
25.2568302154541
35.020511627197266
20.193214416503906
27.74497413635254
28.59733772277832
21.143543243408203
29.63538360595703
16.521961212158203
23.626705169677734
24.38750648498535
18.018014907836914
25.54187774658203
14.184329986572266
19.81690788269043
20.1954402923584
14.772269248962402
21.095888137817383
12.06137752532959
16.4330997467041
16.258766174316406
12.102020263671875
16.905454635620117
9.994129180908203
13.413629531860352
13.031983375549316
10.060772895812988
13.20270824432373
7.827132701873779
10.378826141357422
9.761575698852539
8.270450592041016
9.794621467590332
5.8198747634887695
7.79997444152832
6.743044376373291
6.6092939376831055
7.004265308380127
3.8502144813537598
5.447595119476318
3.7866101264953613
5.470849990844727
5.0052456

In [16]:
DGL.ModelClassifyNodes(model, test_data.dataset)

{'alllabels': [[3, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4],
  [4, 5, 4, 10, 2, 7, 1, 8, 3, 0, 9],
  [0, 8, 9, 1, 4, 10, 2, 10, 7, 4, 5, 3],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 7, 1],
  [8, 1, 9, 4, 3, 4, 0, 10, 3, 4, 6, 4, 2, 7, 5, 10]],
 'allpredictions': [[4, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4],
  [3, 5, 4, 10, 2, 7, 1, 8, 4, 0, 9],
  [0, 8, 9, 1, 4, 10, 2, 10, 7, 3, 5, 4],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 7, 1],
  [8, 1, 9, 4, 3, 4, 10, 10, 3, 4, 6, 4, 2, 7, 5, 10]],
 'trainlabels': [[3, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6],
  [4, 5, 4, 10, 2, 7, 1, 8],
  [0, 8, 9, 1, 4, 10, 2, 10, 7],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6],
  [8, 1, 9, 4, 3, 4, 0, 10, 3, 4, 6, 4]],
 'trainpredictions': [[4, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6],
  [3, 5, 4, 10, 2, 7, 1, 8],
  [0, 8, 9, 1, 4, 10, 2, 10, 7],
  [3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6],
  [8, 1, 9, 4, 3, 4, 10, 10, 3, 4, 6, 4]],
 'validationlabels': [[0], [3], [4], [10], [2]],
 'validationpredictions': [[0], [4], [3], [10], [2]],
 'testlabels'

In [17]:
dictionary = DGL.ModelClassifyNodes(model, test_data.dataset)
# actual = dictionary['alllabels']
actual = []
for l in dictionary['alllabels']:
    actual.extend(l)
    
print(actual)
# predicted = dictionary['allpredictions']
predicted = []
for l in dictionary['allpredictions']:
    predicted.extend(l)
print(predicted)
result = DGL.Accuracy(actual=actual, predicted=predicted)
print(result)
print("Accuracy:", result['accuracy'])

[3, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4, 4, 5, 4, 10, 2, 7, 1, 8, 3, 0, 9, 0, 8, 9, 1, 4, 10, 2, 10, 7, 4, 5, 3, 3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 7, 1, 8, 1, 9, 4, 3, 4, 0, 10, 3, 4, 6, 4, 2, 7, 5, 10]
[4, 4, 7, 4, 2, 3, 10, 1, 5, 8, 6, 0, 9, 4, 3, 5, 4, 10, 2, 7, 1, 8, 4, 0, 9, 0, 8, 9, 1, 4, 10, 2, 10, 7, 3, 5, 4, 3, 0, 5, 8, 4, 4, 3, 2, 4, 9, 6, 10, 7, 1, 8, 1, 9, 4, 3, 4, 10, 10, 3, 4, 6, 4, 2, 7, 5, 10]
{'accuracy': 0.910448, 'correct': 61, 'mask': [False, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True], 'size': 67, 'wrong': 6}
Accuracy: 0.910448


In [18]:
cf = DGL.ConfusionMatrix(actual = actual, predicted = predicted)
print(cf)
fig = Plotly.FigureByConfusionMatrix(cf)
Plotly.Show(fig)

[[ 4  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  2  0  0  0  0  0  0]
 [ 0  0  0  3 12  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  0  0  0  0  0  0  0  5  0]
 [ 1  0  0  0  0  0  0  0  0  0  7]]
